In [1]:
import numpy as np
import sys
if '/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

import cv2
import random
import os
import numpy as np
import json
from matplotlib import pyplot as plt
from PIL import Image
from pycococreatortools import pycococreatortools
from IPython import embed
from tqdm import tqdm
from IPython.core.display import display, HTML
import os
import shutil
import glob 
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer, ColorMode

# labels = ['chair', 'cushion', 'door', 'indoor-plant', 'table', 'vase']
# labels= ['chair', 'cushion', 'door', 'lamp', 'sofa', 'table']
labels = ['chair', 'cushion', 'door', 'indoor-plant', 'sofa', 'table']
semantic_json_root = '/checkpoint/apratik/ActiveVision/active_vision/info_semantic'

class CocoCreator:
    # Assumes root_data_dir has both te GT and propagated segmentation labels
    def __init__(self, root_data_dir, semantic_json_root, labels, segm_dir):
        self.rdd = root_data_dir
        self.sjr = semantic_json_root
        self.labels = labels
        self.segm_dir = segm_dir
        sorted(self.labels)
#         print(f"Init CocoCreator for labels {self.labels}")
        
    def create_coco(self, scene, coco_file_name, pct, segmids):
        hsd = self.load_semantic_json(scene)
        self.create_metadata(hsd)
        self.create_annos(hsd, scene, pct, segmids)
        self.save_json(coco_file_name)
        self.save_visual_dataset(coco_file_name, scene)
    
    def save_visual_dataset(self, coco_file_name, scene):
        DatasetCatalog.clear()
        MetadataCatalog.clear()

        register_coco_instances('foobar', {}, coco_file_name, os.path.join(self.rdd, 'rgb'))
        MetadataCatalog.get('foobar')
        dataset_dicts = DatasetCatalog.get('foobar')
        
        save_dir = os.path.join(self.segm_dir, 'coco_visuals')
        print(f'save_dir {save_dir}')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        for d in dataset_dicts:
            img = cv2.imread(d["file_name"])
            x = d['file_name'].split('/')[-1]
            print(f"filename {d['file_name'], x}, visual_file {os.path.join(save_dir, x)}")
            visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get('foobar'), scale=0.5)
            vis = visualizer.draw_dataset_dict(d)
            img = vis.get_image()
            cv2.imwrite(os.path.join(save_dir, x), img)
#             plt.figure(figsize=(12 , 8))
#             plt.imshow(img)
#             plt.show()
        
    def save_json(self, coco_file_name):
        coco_output = {
            "info": self.INFO,
            "licenses": self.LICENSES,
            "categories": self.CATEGORIES,
            "images": self.IMAGES,
            "annotations": self.ANNOTATIONS,
        }
        
        print(f"Dumping {len(coco_output['annotations'])} annotations to {coco_file_name}")
        with open(coco_file_name, "w") as output_json:
            json.dump(coco_output, output_json)
    
    def create_annos(self, hsd, scene, pct, segmids):
        coco_img_id = -1
        count = 0
        segm_dir = self.segm_dir
        print(f"Scene {scene}, seg dir {segm_dir}")       
        img_dir = os.path.join(self.rdd, 'rgb')
        fs = self.get_segm_files(segm_dir, pct)
        print(f"Creating COCO annotations for {len(fs)} images \n img_dir {img_dir}")
        
#         qfs = []
#      
            
        print(f'{len(fs)} files qualified')
        
        for f in tqdm(fs):
            image_id = int(f.split('.')[0])
            try:
                prop_path = os.path.join(segm_dir, "{:05d}.npy".format(image_id))
                annot = np.load(prop_path).astype(np.uint32)
            except Exception as e:
                print(e)
                continue


            img_filename = "{:05d}.jpg".format(image_id)            
            img = Image.open(os.path.join(img_dir, img_filename))

            # COCO ID and names
            coco_img_id += 1

            image_info = pycococreatortools.create_image_info(
                coco_img_id, os.path.basename(img_filename), img.size
            )

            self.IMAGES.append(image_info)
        
            # for each annotation add to coco format
            for i in np.sort(np.unique(annot.reshape(-1), axis=0)):
                try:
                    if hsd["id_to_label"][i] < 1 or self.label_id_dict[hsd["id_to_label"][i]] not in labels:# or hsd["id_to_label"][i] not in self.label_id_dict:
                        continue
                    category_info = {"id": self.new_old_id[hsd["id_to_label"][i]], "is_crowd": False}
#                     print(f'category_info {category_info}')
                except:
                    continue

                binary_mask = (annot == i).astype(np.uint32)

                annotation_info = pycococreatortools.create_annotation_info(
                    count, coco_img_id, category_info, binary_mask, img.size, tolerance=2
                )
                if annotation_info is not None:
                    self.ANNOTATIONS.append(annotation_info)
                    count += 1
        
        
    def create_metadata(self, hsd):
        self.INFO = {}
        self.LICENSES = [{}]
        self.CATEGORIES = []
        self.IMAGES = []
        self.ANNOTATIONS = []
        
        self.label_id_dict = {}
        self.new_old_id = {}
        idc = 1
        for obj_cls in hsd["classes"]:
            if obj_cls["name"] in self.labels:
                self.CATEGORIES.append({"id": idc, "name": obj_cls["name"], "supercategory": "shape"})
                self.label_id_dict[obj_cls["id"]] = obj_cls["name"]
                self.new_old_id[obj_cls['id']] = idc
                idc += 1
        
        
    def load_semantic_json(self, scene):
        replica_root = '/datasets01/replica/061819/18_scenes'
        habitat_semantic_json = os.path.join(replica_root, scene, 'habitat', 'info_semantic.json')
#         habitat_semantic_json = os.path.join(self.sjr, scene + '_info_semantic.json')
#         print(f"Using habitat semantic json {habitat_semantic_json}")
        with open(habitat_semantic_json, "r") as f:
            hsd = json.load(f)
        if hsd is None:
            print("Semantic json not found!")
        return hsd
        
    def get_segm_files(self, segm_dir, pct):
        cs = [os.path.basename(x) for x in glob.glob(os.path.join(segm_dir, '*.npy'))]
        cs.sort()
        frq = 1/pct
        fs = []
        for x in range(0, len(cs), int(frq)):
            fs.append(cs[x])
        return fs 

def run_coco(out_dir, img_dir,pct, coco_name, segmids):
    cbase = CocoCreator(img_dir, semantic_json_root, labels, out_dir)
    cbase.create_coco(
        scene='apartment_0', 
        coco_file_name=os.path.join(out_dir, coco_name),
        pct=pct,
        segmids=segmids,
    )

In [3]:
# Create val set

test_json = '/checkpoint/apratik/finals/jsons/active_vision/frlapt1_20n0.json'
test_dir = '/checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/'
seg_dir = '/checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/seg'

# val_json0 = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200/83/seg/coco_train.json'
# img_dir_val0 = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200/83/'
# out_dir = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200/83/seg/'


run_coco(seg_dir, test_dir, 0.05, f'cvpr_test5.json', [])

  0%|          | 1/267 [00:00<00:41,  6.34it/s]

Scene apartment_0, seg dir /checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/seg
Creating COCO annotations for 267 images 
 img_dir /checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/rgb
267 files qualified


100%|██████████| 267/267 [00:44<00:00,  6.06it/s]


Dumping 1423 annotations to /checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/seg/cvpr_test5.json
1423 1423
save_dir /checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/seg/coco_visuals
filename ('/checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/rgb/00000.jpg', '00000.jpg'), visual_file /checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/seg/coco_visuals/00000.jpg
filename ('/checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/rgb/00020.jpg', '00020.jpg'), visual_file /checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/seg/coco_visuals/00020.jpg
filename ('/checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/rgb/00040.jpg', '00040.jpg'), visual_file /checkpoint/apratik/ActiveVision/activ

In [10]:
# img_dir = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200/83/'
# out_dir = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200/83/seg'



for traj in [1,2,3,4]:
    img_dir = f'/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/{traj}/default'
    out_dir = f'/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/{traj}/default/seg'

    # ids = [294, 293, 292, 291, 312, 313, 314, 315, 597, 596, 595, 594, 615, 616, 617, 618, 900, 899, 898, 897, 918, 919, 920, 921, 1203, 1202, 1201, 1200, 1221, 1222, 1223, 1224, 1506, 1505, 1504, 1503, 1524, 1525, 1526, 1527]
    gt = [303, 606, 909, 1212, 1515]
    for p in [0,2,4,6,8]:
        ids = [y for x in gt for y in range(x-p, x+p+1) ]
        print(len(ids), (2*p + 1) * 5, ids)
        run_coco(out_dir, img_dir, 1, f'coco_gt_{p}.json', ids)


  0%|          | 0/5 [00:00<?, ?it/s]

5 5 [303, 606, 909, 1212, 1515]
Scene apartment_0, seg dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg
['00000.npy', '00001.npy', '00002.npy']
Creating COCO annotations for 5 images 
 img_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb


100%|██████████| 5/5 [00:00<00:00,  6.08it/s]


Dumping 19 annotations to /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_gt_0.json
19 19
save_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_visuals
filename ('/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb/00303.jpg', '00303.jpg'), visual_file /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_visuals/00303.jpg
filename ('/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb/00606.jpg', '00606.jpg'), visual_file /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_visuals/00606.jpg
filename ('/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb/00909.jpg', '00909.jpg'), visual_file /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_n

  4%|▍         | 1/24 [00:00<00:03,  6.09it/s]

...

Dumping 233 annotations to /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_gt_6.json
233 233
save_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_visuals
filename ('/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb/00297.jpg', '00297.jpg'), visual_file /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_visuals/00297.jpg
filename ('/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb/00298.jpg', '00298.jpg'), visual_file /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_visuals/00298.jpg
filename ('/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb/00299.jpg', '00299.jpg'), visual_file /checkpoint/apratik/data_devfair0187/apartment_0/straightline/n

  1%|▏         | 1/78 [00:00<00:13,  5.82it/s]

['00000.npy', '00001.npy', '00002.npy']
Creating COCO annotations for 78 images 
 img_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb


100%|██████████| 78/78 [00:13<00:00,  6.00it/s]


Dumping 311 annotations to /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_gt_8.json
311 311
save_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_visuals
filename ('/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb/00295.jpg', '00295.jpg'), visual_file /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_visuals/00295.jpg
filename ('/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb/00296.jpg', '00296.jpg'), visual_file /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_visuals/00296.jpg
filename ('/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb/00297.jpg', '00297.jpg'), visual_file /checkpoint/apratik/data_devfair0187/apartment_0/straightline/n